**Imports**

In [ ]:
import numpy as np
import tensorflow as tf
import math
import random


**ID3 Class**

In [ ]:
class ID3:

    class Node:
        
        def __init__(self, feature_index=None, left=None, right=None, class_=None):
            self.feature_index = feature_index
            self.left = left
            self.right = right
            self.class_ = class_

    # Creates a new ID3 Decision Tree classifier.
    def __init__(self, max_depth=20, min_samples_split=3):
        self.root = None
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        
    def fit(self, data_frame, classes, features):
        """
        Fits the ID3 Decision Tree classifier to the provided data.

        Args:
            data_frame: The input data as a DataFrame.
            labels (array): The labels for the input data.
            features (array): The features to be used for building the tree.
        """
        self.root = self.build_tree(data_frame, classes, features, self.max_depth, self.min_samples_split)

    def build_tree(self, train_dataframe, classes, features, max_depth, min_examples, default_class=None):
        """
        Builds the ID3 Decision Tree recursively.

        Args:
            train_dataframe: The input data as a DataFrame.
            classes (array): The labels for the input data.
            features (array): The features to be used for building the tree.
            max_depth (int): The maximum depth of the tree.
            min_examples (int): The minimum number of samples required to split a node.
            default_class: The default class value to be assigned if a leaf node is reached.

        Returns:
            Node: The root node of the built tree.
        """
        pos_percentage, neg_percentage = self.calculate_class_percentages(train_dataframe)

        if max_depth == 0:
            return self.Node(class_=1 if pos_percentage > neg_percentage else 0 if pos_percentage < neg_percentage else default_class)

        if pos_percentage == 1:
            return self.Node(class_=1)
        if neg_percentage == 1:
            return self.Node(class_=0)

        if pos_percentage >= 0.90 and neg_percentage < 0.10:
            return self.Node(class_=1)
        if pos_percentage < 0.10 and neg_percentage >= 0.90:
            return self.Node(class_=0)

        if len(train_dataframe) == 0 or len(features) == 0:
            return self.Node(class_=default_class)

        default_class = 1 if pos_percentage > neg_percentage else 0 if pos_percentage < neg_percentage else random.randint(0, 1)
        b_feature_index = self.find_max_ig_feature(features, classes)

        new_features = np.delete(features, b_feature_index, axis=1)

        left_nodes = train_dataframe[train_dataframe[:, b_feature_index] == 1]
        left_classes = left_nodes[:, -1]
        left_features = left_nodes[:, :-1]

        right_nodes = train_dataframe[train_dataframe[:, b_feature_index] == 0]
        right_classes = right_nodes[:, -1]
        right_features = right_nodes[:, :-1]

        if len(left_nodes) < min_examples and len(right_nodes) < min_examples:
            return self.Node(class_=default_class)

        left_tree = self.build_tree(left_nodes, left_classes, left_features, max_depth - 1, min_examples, default_class)
        right_tree = self.build_tree(right_nodes, right_classes, right_features, max_depth - 1, min_examples, default_class)

        return self.Node(b_feature_index, left_tree, right_tree)

    
    def predict(self, root, test):
        """
        Predicts the class label for the given test data.

        Args:
            root (Node): The root node of the decision tree.
            test (array): The input test data.

        Returns:
            int: The predicted class label.
        """

        # If the current node is None, return None (base case)
        if root is None:
            return None

        # If the current node is a leaf node, return the assigned class
        if root.class_ is not None:
            return root.class_

        # Get the feature index to check from the current node
        feature_index = root.feature_index

        # Check the value of the feature in the test data
        value = test[feature_index]

        # Traverse the left subtree if the feature value is 1
        if value == 1:
            return self.predict(root.left, test)

        # Traverse the right subtree if the feature value is 0
        elif value == 0:
            return self.predict(root.right, test)



    def calculate_class_percentages(self, data_frame):
        """
        Calculates the percentages of positive and negative labels in a dataset.

        Args:
            data_frame (array-like): The input data.

        Returns:
            tuple: A tuple containing the percentage of positive labels and the percentage of negative labels.
        """

        pos_percentage = 0
        neg_percentage = 0

        # Extract the class labels from the data_frame
        classes = np.array([df[-1] for df in data_frame])

        # Count the occurrences of positive and negative classes
        pos_count = np.sum(classes == 1)
        neg_count = np.sum(classes == 0)

        all_categories = pos_count + neg_count

        # Calculate the percentages of positive and negative classes
        if all_categories != 0:
            pos_percentage = pos_count / all_categories
            neg_percentage = neg_count / all_categories

        # Return the percentages of positive and negative classes
        return pos_percentage, neg_percentage

    
    def find_max_ig_feature(self, features, classes):
        """
        Finds the feature with the maximum information gain.

        Args:
            features (array): The features to be used for calculating information gain.
            classes (array): The labels for the input data.

        Returns:
            int: The index of the feature with the maximum information gain.
        """

        # Calculate the overall entropy based on the classes
        overall_entropy = self.calculate_entropy(classes)

        # List to store the calculated information gains for each feature
        igs = []

        # Iterate over each feature index
        for i in range(len(features[0])):
            # Get the i-th column of every feature
            feature_i = features[:, i]

            # Calculate the information gain for this specific feature
            ig_feat_i = self.calculate_ig(classes, feature_i, overall_entropy)
            igs.append(ig_feat_i)

        # Return the index of the feature with the maximum information gain
        return np.argmax(igs)

    
    def calculate_entropy(self, classes):
        """
        Calculates the entropy of a set of labels.

        Args:
            classes (array): The labels for the input data.

        Returns:
            float: The entropy.
        """

        # Get the unique classes in the array
        unique_classes = np.unique(classes)

        entropy = 0
        # Calculate the entropy for each class
        for c in unique_classes:
            # Count the number of times the class occurs
            class_count = np.count_nonzero(classes == c)
            # Calculate the probability of the class
            p_c = class_count / len(classes)
            # Calculate the entropy for this class
            entropy += -p_c * math.log(p_c, 2)

        return entropy

        
    def calculate_ig(self, classes, feature, entropy):
        """
        Calculates the information gain for a given feature.

        Args:
            classes (array): The labels for the input data.
            feature (array): The feature to be used for calculating information gain.
            entropy (float): The entropy of the labels.

        Returns:
            float: The information gain.
        """

        unique_classes = np.unique(classes)
        unique_feature_values = np.unique(feature)

        Hc_feature = 0
        # Calculate the probability of each unique feature value
        pf = np.bincount(feature) / len(feature)

        # Iterate over the unique feature values
        for feat in unique_feature_values:
            # Find the indices of the feature values that match the current value
            indices = np.where(feature == feat)[0]
            # Get the corresponding class values for the feature values that match the current value
            classes_of_feat = classes[indices]
            for c in unique_classes:
                # Calculate the probability of class c given feature value feat
                pcf = np.count_nonzero(classes_of_feat == c) / len(classes_of_feat)
                if pcf != 0:
                    temp_H = -pf[feat] * pcf * math.log(pcf, 2)
                    Hc_feature += temp_H

        ig = entropy - Hc_feature
        return ig
   

    def score_tree(self, test_dataframe):
        """
        Calculates the accuracy, precision, recall, and F1 score of the decision tree on a test dataset.

        Args:
            test_dataframe (array): The test dataset.

        Returns:
            float: The accuracy of the decision tree.
            float: The precision of the decision tree.
            float: The recall of the decision tree.
            float: The F1 score of the decision tree.
        """

        true_positives = 0
        true_negatives = 0
        false_positives = 0
        false_negatives = 0

        for test in test_dataframe:
            # Getting the tree's prediction for the specific test (where test is the review of the movie in binary)
            prediction = self.predict(self.root, test)
            # The actual class of the review stored in the test dataframe
            actual_class = test[-1]

            if actual_class == 1 and prediction == 1:
                true_positives += 1
            elif actual_class == 0 and prediction == 0:
                true_negatives += 1
            elif actual_class == 1 and prediction == 0:
                false_negatives += 1
            elif actual_class == 0 and prediction == 1:
                false_positives += 1

        # Calculating the results (accuracy, precision, recall, f1 score)
        accuracy = round((true_positives + true_negatives) / (true_negatives + true_positives + false_positives + false_negatives), 4)
        precision = round(true_positives / (true_positives + false_positives), 4)
        recall = round(true_positives / (true_positives + false_negatives), 4)
        f1_score = round((2 * precision * recall) / (precision + recall), 4)

        return accuracy, precision, recall, f1_score


**Creating the binary vector data for both training testing, by loading the Keras IMDb Dataset.**

In [ ]:
def create_bin_data(m, n, k, test=0):
    """
    Creates the binary vector data for training or testing.

    Args:
        m (int): The number of most frequent words to be included as features.
        n (int): The number of most frequent words to be skipped.
        k (int): The number of rarest words to be skipped.
        test (int, opt): Indicator for creating test data. Defaults to 0.

    Returns:
        numpy array: The binary vector data.
        numpy array: The class labels.
        numpy array: The features.
    """

    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=m - k, skip_top=n)

    word_index = tf.keras.datasets.imdb.get_word_index()
    index2word = {i + 3: word for word, i in word_index.items()}
    index2word[0] = '[pad]'
    index2word[1] = '[bos]'
    index2word[2] = '[oov]'
    x_train = np.array([' '.join([index2word[idx] for idx in text]) for text in x_train])
    x_test = np.array([' '.join([index2word[idx] for idx in text]) for text in x_test])

    data_frame = []
    
    if test == 0:
        vocabulary = np.array([word for text in x_train for word in text.split()])
        vocabulary = np.unique(vocabulary)
        for text, class_ in zip(x_train, y_train):
            splitted_text = text.split()
            features = np.isin(vocabulary, splitted_text).astype(bool)
            data_frame.append(np.concatenate([features, [class_]]))
        data_frame = np.array(data_frame)
    else:
        vocabulary = np.array([word for text in x_test for word in text.split()])
        vocabulary = np.unique(vocabulary)
        for text, class_ in zip(x_test, y_test):
            splitted_text = text.split()
            features = np.isin(vocabulary, splitted_text).astype(bool)
            data_frame.append(np.concatenate([features, [class_]]))
        data_frame = np.array(data_frame)

    # Extracting the class labels
    classes = data_frame[:, -1]
    # Extracting the features
    features = data_frame[:, :-1]

    print("Data is now read!")

    return data_frame, classes, features

**Main**

In [ ]:
if __name__ == "__main__":
    
    print("Creating binary data...")
    data_frame, classes, features = create_bin_data(m=850, n=45, k=1)
    test_data_frame, test_classes, test_features = create_bin_data(m=850, n=45, k=1, test=1)

    id3 = ID3()
    id3.fit(data_frame, classes, features)
    accuracy, precision, recall,f1_score = id3.score_tree(test_data_frame)
    
    print("Metrics:")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1_score:.2f}")
